In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import joblib,glob,os,cv2
from scipy.ndimage import filters
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [ ]:
def get_gradient(im_dx, im_dy): 
    assert im_dx.shape == im_dy.shape
    m, n = im_dx.shape
    grad_mag = np.zeros((m, n))
    grad_angle = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            grad_mag[i][j] = np.sqrt(im_dx[i][j] ** 2 + im_dy[i][j] ** 2)
            grad_angle[i][j] = np.arctan2(im_dy[i][j], im_dx[i][j])
    return grad_mag, grad_angle

In [ ]:
def get_angle_bin(angle):
    angle_deg = angle * 180 / np.pi
    if angle_deg==0:
        d1=d2=p1=p2=bit=0
        return 0,bit,p1,p2,d1,d2
    
    
    elif angle_deg==20:
        d1=d2=p1=p2=bit=0
        return 1,bit,p1,p2,d1,d2
    
    
    elif angle_deg==40:
        d1=d2=p1=p2=bit=0
        return 2,bit,p1,p2,d1,d2
    
    
    elif angle_deg==60:
        d1=d2=p1=p2=bit=0
        return 3,bit,p1,p2,d1,d2
    
    
    elif angle_deg==80:
        d1=d2=p1=p2=bit=0
        return 4,bit,p1,p2,d1,d2
    
    
    elif angle_deg==100:
        d1=d2=p1=p2=bit=0
        return 5,bit,p1,p2,d1,d2
    
    
    elif angle_deg==120:
        d1=d2=p1=p2=bit=0
        return 6,bit,p1,p2,d1,d2
    
    
    elif angle_deg==140:
        d1=d2=p1=p2=bit=0
        return 7,bit,p1,p2,d1,d2
    
    
    elif angle_deg==160:
        d1=d2=p1=p2=bit=0
        return 8,bit,p1,p2,d1,d2
    
    
    elif 0 < angle_deg < 20: 
        bit=1
        d1 = abs(angle_deg-0)
        d2 = abs(angle_deg-20)
        p1 = d1/20
        p2 = d2/20
        return 0,bit,p1,p2,d1,d2
    
    
    elif 20 < angle_deg < 40:
        bit=1
        d1 = abs(angle_deg-20)
        d2 = abs(angle_deg-40)
        p1 = d1/20
        p2 = d2/20
        return 1,bit,p1,p2,d1,d2
    
    
    elif 40 < angle_deg < 60:
        bit=1
        d1 = abs(angle_deg-40)
        d2 = abs(angle_deg-60)
        p1 = d1/20
        p2 = d2/20
        return 2,bit,p1,p2,d1,d2
    
    
    elif 60 < angle_deg < 80:
        bit=1
        d1 = abs(angle_deg-60)
        d2 = abs(angle_deg-80)
        p1 = d1/20
        p2 = d2/20
        return 3,bit,p1,p2,d1,d2
    
    
    elif 80 < angle_deg < 100:
        bit=1
        d1 = abs(angle_deg-80)
        d2 = abs(angle_deg-100)
        p1 = d1/20
        p2 = d2/20
        return 4,bit,p1,p2,d1,d2
    
    
    elif 100 < angle_deg < 120:
        bit=1
        d1 = abs(angle_deg-100)
        d2 = abs(angle_deg-120)
        p1 = d1/20
        p2 = d2/20
        return 5,bit,p1,p2,d1,d2
    
    
    elif 120 < angle_deg < 140:
        bit=1
        d1 = abs(angle_deg-120)
        d2 = abs(angle_deg-140)
        p1 = d1/20
        p2 = d2/20
        return 6,bit,p1,p2,d1,d2
    
    
    elif 140 < angle_deg < 160:
        bit=1
        d1 = abs(angle_deg-140)
        d2 = abs(angle_deg-160)
        p1 = d1/20
        p2 = d2/20
        return 7,bit,p1,p2,d1,d2
    
    
    elif 160 < angle_deg < 180:
        bit=1
        d1 = abs(angle_deg-160)
        d2 = abs(angle_deg-180)
        p1 = d1/20
        p2 = d2/20
        return 8,bit,p1,p2,d1,d2

In [ ]:
def build_histogram(grad_mag, grad_angle, cell_size):
    m, n = grad_mag.shape
    M = int(m / cell_size)
    N = int(n / cell_size)
    ori_histo = np.zeros((M, N, 9))
    for i in range(M):
        for j in range(N):
            for x in range(cell_size):
                for y in range(cell_size):
                    angle = grad_angle[i * cell_size + x][j * cell_size + y]
                    mag = grad_mag[i * cell_size + x][j * cell_size + y]
                    bin, bit, p1, p2, d1, d2 = get_angle_bin(angle)
                    if bit == 0:
                        ori_histo[i][j][bin] += mag
                    else:
                        if bin==8:
                            if d2 > d1:
                                ori_histo[i][j][bin] += max(p1,p2)*mag
                                ori_histo[i][j][0] += min(p1,p2)*mag
                            else:
                                ori_histo[i][j][0] += max(p1,p2)*mag
                                ori_histo[i][j][bin] += min(p1,p2)*mag
                        else:
                            if d2 > d1:
                                ori_histo[i][j][bin] += max(p1,p2)*mag
                                ori_histo[i][j][bin+1] += min(p1,p2)*mag
                            else:
                                ori_histo[i][j][bin+1] += max(p1,p2)*mag
                                ori_histo[i][j][bin] += min(p1,p2)*mag
                        

    return ori_histo

In [ ]:
def get_block_descriptor(ori_histo, block_size):
    M, N, bins = ori_histo.shape
    e = 0.001
    ori_histo_normalized = np.zeros((M - block_size + 1, N - block_size + 1, bins * block_size * block_size))
    for i in range(M - block_size + 1):
        for j in range(N - block_size + 1):
            unnormalized = []
            for x in range(block_size):
                for y in range(block_size):
                    for z in range(bins):
                        unnormalized.append(ori_histo[i + x][j + y][z])
            unnormalized = np.asarray(unnormalized)
            den = np.sqrt(np.sum(unnormalized ** 2) + e ** 2)
            normalized = unnormalized / den
            for p in range(bins * block_size * block_size):
                ori_histo_normalized[i][j][p] = normalized[p]
            return ori_histo_normalized

In [ ]:
data = []
labels = []
human_im_path = '/content/drive/My Drive/Dataset/DATAIMAGE/human/'
nonhuman_im_path = '/content/drive/My Drive/Dataset/DATAIMAGE/nonhuman/'


In [ ]:
# Load the positive features
for filename in glob.glob(os.path.join(human_im_path,"*.png")):
    im = cv2.imread(filename,0)
    im = cv2.resize(im,(64,128))
    im_dx = filters.gaussian_filter(im,(3,3),(1,0))
    im_dy = filters.gaussian_filter(im,(3,3),(0,1))
    grad_mag, grad_angle = get_gradient(im_dx, im_dy)
    ori_histo = build_histogram(grad_mag, grad_angle, 8)
    ori_histo_normalized = get_block_descriptor(ori_histo, 2)
    hog = ori_histo_normalized.reshape((-1))
    data.append(hog)
    labels.append(1)


In [ ]:
len(data)

2426

In [ ]:
ori_histo.shape

(16, 8, 9)

In [ ]:
# Load the positive features
for filename in glob.glob(os.path.join(nonhuman_im_path,"*.jpg")):
    im = cv2.imread(filename,0)
    im = cv2.resize(im,(64,128))
    im_dx = filters.gaussian_filter(im,(3,3),(1,0))
    im_dy = filters.gaussian_filter(im,(3,3),(0,1))
    grad_mag, grad_angle = get_gradient(im_dx, im_dy)
    ori_histo = build_histogram(grad_mag, grad_angle, 8)
    ori_histo_normalized = get_block_descriptor(ori_histo, 2)
    hog = ori_histo_normalized.reshape((-1))
    data.append(hog)
    labels.append(0)

In [ ]:
len(data)

5660

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=0)
print("Labels for training and testing data")
print(len(train_data))
print(len(test_data))

Labels for training and testing data
3961
1698


In [ ]:
print(len(train_labels))
print(len(test_labels))

3961
1698


In [ ]:
hog.shape

(3780,)

In [ ]:
labels = np.unique(train_labels)
print(labels) 

[0 1]


In [ ]:
model = SVC(kernel='linear')

In [ ]:
model.fit(train_data, train_labels)

SVC(kernel='linear')

In [ ]:
predictions = model.predict(test_data)
print(predictions)

[0 0 1 ... 0 1 0]


In [ ]:
percentage = model.score(test_data, test_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
res = confusion_matrix(test_labels, predictions)
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(test_data)}")
print(f"Accuracy = {percentage*100} %")

Confusion Matrix
[[740 232]
 [457 269]]
Test Set: 1698
Accuracy = 59.42285041224971 %
